I just took all of my code from the project and changed it so it works with Google Colab. Just change all of the variables you want and run the cells in order.

You can change these variables.

In [ ]:
tag = 'furry' #Search term
requestedFics = 5 #Number of works to be collected
language = 'English' #Work language
getExplicit = True #If we will collect explicit works
training = True #If we are going to train a new model
numGenerate = 2000 #Number of characters the finished work will have
epochs = 25 #Number of epochs to generate
startString = 'The ' #First word of the generated work

workIds.py

In [ ]:
from bs4 import BeautifulSoup
import requests
import csv
import time

#Variables
pageEmpty = False
baseUrl = ""
url = "https://archiveofourown.org/tags/" + tag + "/works"
recordedFics = 0
csvName = "workIds"

#Keeps track of all work ids to not repeat
seenIds = []

#Must be over 5
delay = 5

##Functions
def getIds(header_info=''):
    global pageEmpty
    headers = {'user-agent' : header_info}
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, "lxml")
    works = soup.select("li.work.blurb.group")

    #See if we collected all available work
    if(len(works) == 0):
        pageEmpty = True

    #Creates list of ids
    ids = []
    for tag in works:
        t = tag.get('id')
        t = t[5:]
        if not t in seenIds:
            ids.append(t)
            seenIds.append(t)
    return ids

def updateNextPage():
    global url
    key = "page="
    start = url.find(key)

    #Checks for a page indicator
    if(start != -1):
        #Finds the indicator in the url
        pageStartIndex = start + len(key)
        pageEndIndex = url.find("&", pageStartIndex)
        #Runs if the indicator is in the middle of the url
        if(pageEndIndex != -1):
            page = int(url[pageStartIndex:pageEndIndex]) + 1
            url = url[:pageStartIndex] + str(page) + url[pageEndIndex:]
        #Runs if the indicator is at the end of the url
        else:
            page = int(url[pageStartIndex:]) + 1
            url = url[:pageStartIndex] + str(page)

    #Since there is no page indicator, we must be on page 1
    else:
        #If there are other modifiers
        if(url.find("?") != -1):
            url = url + "&page=2"
        #If there are no modifiers
        else:
            url = url + "?page=2"

#Writes ids and url to .csv file
def writeIds(ids):
    global recordedFics
    with open(csvName + ".csv", 'a') as csvfile:
        wr = csv.writer(csvfile, delimiter=',')
        for id in ids:
            if(notFinished()):
                wr.writerow([id, url])
                recordedFics = recordedFics + 1
            else:
                break

#Checks if we have too many files or if the page is empty
def notFinished():
    if(pageEmpty):
        return False

    if(requestedFics == 0):
        return True
    else:
        if(recordedFics < requestedFics):
            return True
        else:
            return False

#Runs functions to get the work ids
def processIds(headerInfo=''):
    while(notFinished()):
        #Delay between requests as per AO3's terms of service
        time.sleep(delay)
        ids = getIds(headerInfo)
        writeIds(ids)
        updateNextPage()

#Gets called to start the program
def main():
    #Clears the workIds file
    idFile = open(csvName + ".csv", "w")
    idFile.truncate()
    idFile.close()

    #Checks if the number of requested works has been set
    if(requestedFics == 0):
        print('WARNING! Number of requested works not set. Will collect all available works.\nprocessing...')
    else:
        print ("processing...")
        
    processIds()
    print ("Finished processing")

main()

getFanfics.py

In [ ]:
from bs4 import BeautifulSoup
import requests
import csv
import time
import sys
import os

#Variables
ficIds = [csvName + '.csv']
csvOut = 'works.txt'

##Functions
def getTagInfo(category, meta):
	try:
		tagList = meta.find("dd", class_=str(category) + ' tags').find_all(class_="tag")
	except AttributeError:
		return []
	return [result.text for result in tagList] 
	
#Gets information about works
def getStats(meta):
	#Defines work categories
	categories = ['language', 'published', 'status', 'words', 'chapters', 'comments', 'kudos', 'bookmarks', 'hits'] 
	stats = list(map(lambda category: meta.find("dd", class_=category), categories))

	if not stats[2]:
		stats[2] = stats[1]
	try:		
		stats = [stat.text for stat in stats]
	except AttributeError as e:
		newStats = []
		for stat in stats:
			if stat: newStats.append(stat.text)
			else: newStats.append('null')
		stats = newStats

	stats[0] = stats[0].rstrip().lstrip()
	status = meta.find("dt", class_="status")
	if not status: status = 'Completed' 
	else: status = status.text.strip(':')
	stats.insert(2, status)

	return stats      

#Defines tags and gets their value from function
def getTags(meta):
	tags = ['rating', 'category', 'fandom', 'relationship', 'character', 'freeform']
	return list(map(lambda tag: getTagInfo(tag, meta), tags))

#Gets kudos
def getKudos(meta):
	if(meta):
		#Gets kudos from work
		users = []
		kudos = meta.contents

		#Extracts users from kudos variable
		for kudo in kudos:
			if kudo.name == 'a':
				if 'more users' not in kudo.contents[0] and '(collapse)' not in kudo.contents[0]:
					users.append(kudo.contents[0])
		
		return users
	return []

#Gets bookmarks
def getBookmarks(url, headerInfo):
	bookmarks = []
	headers = {'user-agent' : headerInfo}

	req = requests.get(url, headers=headers)
	src = req.text

	time.sleep(delay)
	soup = BeautifulSoup(src, 'html.parser')

	sys.stdout.write('Scraping bookmarks ')
	sys.stdout.flush()

	#Finds all pages
	if(soup.find('ol', class_='pagination actions')):
		pages = soup.find('ol', class_='pagination actions').findChildren("li" , recursive=False)
		maxPages = int(pages[-2].contents[0].contents[0])
		count = 1
	
		sys.stdout.write('(' + str(maxPages) + ' pages)')
		sys.stdout.flush()

		while count <= maxPages:
			#Extracts bookmarks
			tags = soup.findAll('h5', class_='byline heading')
			bookmarks += getUsers(tags)

			#Goes to next page
			count += 1
			req = requests.get(url+'?page='+str(count), headers=headers)
			src = req.text
			soup = BeautifulSoup(src, 'html.parser')
			sys.stdout.write('.')
			sys.stdout.flush()
			time.sleep(delay)
	else:
		tags = soup.findAll('h5', class_='byline heading')
		bookmarks += getUsers(tags)

	return bookmarks

#Gets users from meta data
def getUsers(meta):
	users = []
	for tag in meta:
			user = tag.findChildren("a" , recursive=False)[0].contents[0]
			users.append(user)

	return users
	
#Runs if the id is invalid
def accessDenied(soup):
	if(soup.find(class_="flash error")):
		return True
	if(not soup.find(class_="work meta group")):
		return True
	return False

#Writes all information into a csv file
def writeCsv(ficId, language, writer, headerInfo=''):
	print('Scraping', ficId)
	url = 'http://archiveofourown.org/works/'+str(ficId)+'?view_adult=true'
	headers = {'user-agent' : headerInfo}
	req = requests.get(url, headers=headers)
	src = req.text
	soup = BeautifulSoup(src, 'html.parser')
	if(accessDenied(soup)):
		print('Access Denied')
		errorRow = ' '.join(ficId) + ' Access Denied'
		print('ERROR: ' + errorRow)
	else:
		meta = soup.find("dl", class_="work meta group")
		tags = getTags(meta)
		stats = getStats(meta)

		#Checks if work is in the correct language
		if language != False and language != stats[0]:
			print('This work is not in ' + language + ', skipping...')
		elif((getExplicit == False) and (tags[0][0] == 'Explicit' or tags[0][0] == 'Mature' or tags[0][0] == 'Not Rated')):
			print('This work is explicit, skiping...')
		else:
			#Gets the work from ao3
			content = soup.find("div", id= "chapters")
			chapters = content.select('p')
			chaptertext = '\n\n'.join([chapter.text for chapter in chapters])
			row = [chaptertext]
			try:
				writer.writerow(row)
			except:
				print('Unexpected error: ', sys.exc_info()[0])
				errorRow = ' '.join(ficId) + ' ' + ' '.join([sys.exc_info()[0]])
				print('ERROR: ' + errorRow)
			print('Done.')

#This is called to start the program
def main():
	os.chdir(os.getcwd())
	with open(csvOut, 'w') as f_out:
		writer = csv.writer(f_out)
		csvFname = ficIds[0]
		with open(csvFname, 'r+') as f_in:
			reader = csv.reader(f_in)
			for row in reader:
				if not row:
					continue
				writeCsv(row[0], language, writer)
				time.sleep(delay)
			
			print('Finished scraping')

main()

prepWorks.py

In [ ]:
import re

#Variables
textFile = csvOut
textOut = 'prepWork.txt'
textContent = ''

##Functions
def main():
	global textContent
	#List of things to replace with the list below them, replaceList[0] with be replaced with replaceTerms[0]
	replaceList = ['(See the end of the chapter for  notes.)', '(See the end of the chapter for notes.)', '","', '"New work', '""']
	replaceTerms = ['', '', '', '\n\n', '"']

	#Opens both text files
	with open(textFile) as txtIn, open(textOut, 'w+') as txtOut:
		#Fixes formating that applies to each line
		x=0
		for line in txtIn:
			#Skips if it's the first time running, removes the first line
			if(x==0):
				x=x+1
				continue
			#Replace List
			i=0
			for word in replaceList:
				line = line.replace(word, replaceTerms[i])
				i = i+1
			#Removes empty lines
			if(line.isspace()):
				line = ''

			textContent = textContent + line

		#Fixes formating that applies to the entire file
		textContent = re.sub("\ +", " ", textContent)
		textContent = re.sub("\\n+", "\n\n", textContent)
		txtOut.write(textContent)
		print('Finished preparing')
main()

createWork.py

In [ ]:
import tensorflow as tf
import numpy as np
import os

##Variables
#File location of the loaded text converted to absolute path
prepWork = os.path.abspath(textOut)

#Training constants
checkpointDir = './training_checkpoints'
seqLenth = 100
batchSize = 64
bufferSize = 10000
embeddingDim = 256
rnnUnits = 1024
temperature = 1.0

#Finished work name
try:
	reqFics = sum(1 for line in open(csvName + '.csv')) // 2
except:
	reqFics = csvName + '.csv'

##Functions
#Saves the generated text into a file
def genFile(text):
	workName = 'epochs{}_reqFics{}'.format(epochs,reqFics)

	txtOut = open(workName + '.txt', 'w')
	txtOut.write(text)
	txtOut.close()

#This is called to start the program
def main():
	#Encodes text file into utf-8
	text = open(prepWork, 'rb').read().decode(encoding='utf-8')

	#Gets list of known characters
	vocab = sorted(set(text))
	print('{} unique characters'.format(len(vocab)))

	#Converts readable text to machine text and vis versa
	char2idx = {unique:idx for idx, unique in enumerate(vocab)}
	idx2char = np.array(vocab)

	#Converts text to a number that represents it
	textAsInt = np.array([char2idx[char] for char in text])
	charDataset = tf.data.Dataset.from_tensor_slices(textAsInt)
	sequences = charDataset.batch(seqLenth+1, drop_remainder=True)

	def splitInputTarget(chunk):
		inputText = chunk[:-1]
		targetText = chunk[1:]
		return inputText, targetText

	dataset = sequences.map(splitInputTarget)
	dataset = dataset.shuffle(bufferSize).batch(batchSize, drop_remainder=True)

	vocabSize = len(vocab)

	#Prepares the training of a model. This can train a new model or load a previously trained one
	def buildModel(vocabSize, embeddingDim, rnnUnits, batchSize):
		model = tf.keras.Sequential([
			tf.keras.layers.Embedding(vocabSize, embeddingDim, batch_input_shape=[batchSize, None]),
			tf.keras.layers.GRU(rnnUnits, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
			tf.keras.layers.Dense(vocabSize)
		])
		return model

	#This will run when the program is creating a trained model
	if(training == True):
		model = buildModel(vocabSize=len(vocab), embeddingDim=embeddingDim, rnnUnits=rnnUnits, batchSize=batchSize)

		def loss(labels, logits):
			return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

		model.compile(optimizer='adam', loss=loss)

		checkpointPrefix = os.path.join(checkpointDir, 'chkpt_{epoch}')
		checkpointCallback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpointPrefix, save_weights_only=True)

		history = model.fit(dataset, epochs=epochs, callbacks=[checkpointCallback])

	#Collects information about the trained model and presents them
	model = buildModel(vocabSize, embeddingDim, rnnUnits, batchSize=1)
	model.load_weights(tf.train.latest_checkpoint(checkpointDir))
	model.build(tf.TensorShape([1, None]))
	model.summary()

	#Final code that generates the output work
	def generateText(model, startString):
		inputEval = [char2idx[s] for s in startString]
		inputEval = tf.expand_dims(inputEval, 0)

		textGenerated = []

		model.reset_states()
		for i in range(numGenerate):
			predictions = model(inputEval)
			predictions = tf.squeeze(predictions, 0)

			#Predictions get crazier as temperature goes up
			predictions = predictions / temperature
			predictedId = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

			inputEval = tf.expand_dims([predictedId], 0)
			textGenerated.append(idx2char[predictedId])

		return (startString + ''.join(textGenerated))

	#Gets generated text and runs the save function
	genText = generateText(model, startString=startString)
	genFile(genText)
	print('Finished generating work')

main()